In [3]:
# Loading and cleaning the uploaded CSV for the user
# This code will:
# 1. Read the CSV file at /mnt/data/ieg_world_bank_project_performance_ratings_10-16-2025.csv
# 2. Inspect columns, data types, missingness, and basic stats
# 3. Drop obviously non-informative columns (constant columns, columns with mostly missing values, obvious IDs/urls)
# 4. Remove duplicate rows and normalize column names (strip, lower)
# 5. Output the cleaned dataframe and save it to /mnt/data/cleaned_world_bank.csv
# The reasoning for dropped columns will be printed.

import pandas as pd

input_path = "ieg_world_bank_project_performance_ratings_10-16-2025.csv"
df = pd.read_csv(input_path, low_memory=False)

# Inspect
info = {
    "num_rows": len(df),
    "num_columns": df.shape[1],
    "columns": df.columns.tolist(),
    "dtypes": df.dtypes.apply(lambda x: str(x)).to_dict(),
    "missing_perc": (df.isna().mean()*100).round(2).to_dict(),
    "n_unique": df.nunique(dropna=False).to_dict(),
}

print("Initial dataset info:")
print(f"Rows: {info['num_rows']}, Columns: {info['num_columns']}")
print("\nColumns and dtypes:")
for c, t in info["dtypes"].items():
    print(f" - {c}: {t}  (missing%: {info['missing_perc'][c]}  unique: {info['n_unique'][c]})")

# Normalize column names: strip and lower for consistent handling, keep mapping to original
orig_cols = df.columns.tolist()
norm_map = {c: c.strip().lower().replace(" ", "_") for c in orig_cols}
df = df.rename(columns=norm_map)

# Decide which columns to drop automatically:
to_drop = []

# 1) drop columns that are constant (single unique non-null value)
for col in df.columns:
    nunique = df[col].nunique(dropna=True)
    if nunique <= 1:
        to_drop.append(col)

# 2) drop columns with very high missingness (>=60%)
high_missing_thresh = 0.60
missing_frac = df.isna().mean()
for col in df.columns:
    if missing_frac[col] >= high_missing_thresh and col not in to_drop:
        to_drop.append(col)

# 3) drop obvious identifiers / urls / notes that are text heavy and unlikely to be "economic indicators"
# Heuristics: column names containing "id", "url", "link", "notes", "description", "comment", "source", "code"
identifier_keywords = ["id", "url", "link", "notes", "note", "description", "comment", "source", "code", "hash"]
for col in df.columns:
    low = col.lower()
    if any(kw in low for kw in identifier_keywords):
        # but avoid dropping 'country' or 'year'
        if not any(k in low for k in ["country", "year"]):
            if col not in to_drop:
                to_drop.append(col)

# 4) drop columns with extremely high cardinality relative to rows (likely unique identifiers)
for col in df.columns:
    if col in to_drop or col in ("country", "year"):
        continue
    if df[col].nunique(dropna=False) / max(1, len(df)) > 0.9:
        # skip numeric continuous indicators that may be naturally varied; use name heuristics
        if not any(k in col for k in ["gdp", "inflation", "unemployment", "rate", "percent", "score", "rating", "year"]):
            to_drop.append(col)

# Ensure we don't accidentally drop the most likely economic columns: keep columns named like country, year, gdp, inflation, unemployment
protected_keywords = ["country", "year", "gdp", "inflation", "unemployment", "unemployment_rate", "population", "exchange", "rate"]
to_drop = [c for c in to_drop if not any(pk in c for pk in protected_keywords)]

print("\nAutomatically selected columns to drop (by heuristic):")
for c in to_drop:
    print(" -", c)

# Apply drops
df_clean = df.drop(columns=to_drop, errors="ignore").copy()

# Additional cleaning: strip whitespace for object columns, convert year-like to int if possible
for col in df_clean.select_dtypes(include=["object"]).columns:
    df_clean[col] = df_clean[col].astype(str).str.strip().replace({"nan": pd.NA})

# Try convert 'year' to integer if present
year_candidates = [c for c in df_clean.columns if "year" in c]
for yc in year_candidates:
    try:
        df_clean[yc] = pd.to_numeric(df_clean[yc], errors="coerce").astype("Int64")
    except Exception:
        pass

# Drop duplicate rows
before_dup = len(df_clean)
df_clean = df_clean.drop_duplicates()
after_dup = len(df_clean)

print(f"\nDropped {before_dup - after_dup} duplicate rows.")

# Save cleaned file
output_path = "cleaned_world_bank.csv"
df_clean.to_csv(output_path, index=False)

print(f"\nCleaned dataset saved to: {output_path}")

# Show top rows for user to inspect (first 50 rows to respect space)
display_df = df_clean.head(50)

# Also provide a compact summary table of remaining columns
summary = pd.DataFrame({
    "column": df_clean.columns,
    "dtype": [str(t) for t in df_clean.dtypes],
    "missing_pct": (df_clean.isna().mean()*100).round(2).values,
    "n_unique": df_clean.nunique(dropna=False).values
})
print("\nSummary of remaining columns:")

# Print a final text block for the user explaining the actions taken
print("\nSummary of cleaning actions taken:")
print("- Normalised column names (trimmed, lowercased, spaces replaced with underscores).")
print(f"- Removed {len(to_drop)} columns automatically because they were constant, had >={int(high_missing_thresh*100)}% missing values, or appeared to be identifiers/notes/URLs according to heuristics.")
print("- Stripped whitespace in text columns and attempted to convert 'year' columns to integers when possible.")
print("- Removed duplicate rows.")
print(f"- Saved the cleaned dataset to: {output_path}")

# If the dataframe contains obvious economic indicator columns, list them as likely usable attributes
likely_indicators = [c for c in df_clean.columns if any(k in c for k in ["gdp","inflation","unemployment","rate","percent","score","rating","population","gni","income"])]
if likely_indicators:
    print("\nLikely economic indicator columns detected (based on name heuristics):")
    for c in likely_indicators:
        print(" -", c)
else:
    print("\nNo obvious GDP/inflation/unemployment columns were detected by simple name heuristics. You may want to provide or merge a World Bank indicators dataset (e.g., GDP, inflation, unemployment) if this file is a different table (e.g., project ratings).")

# Provide download link
print(f"\n[Download cleaned CSV](/mnt/data/cleaned_world_bank.csv)")


Initial dataset info:
Rows: 11546, Columns: 21

Columns and dtypes:
 - As of Date: object  (missing%: 0.0  unique: 1)
 - Project ID: object  (missing%: 0.0  unique: 11546)
 - Project Name: object  (missing%: 0.0  unique: 10158)
 - WB Region: object  (missing%: 0.0  unique: 9)
 - Country / Economy: object  (missing%: 0.51  unique: 182)
 - Country / Economy Lending Group: object  (missing%: 0.0  unique: 4)
 - Country / Economy FCS Status: object  (missing%: 55.46  unique: 3)
 - Country / Economy FCS Lending Group: object  (missing%: 0.0  unique: 7)
 - Practice Group: object  (missing%: 0.0  unique: 6)
 - Global Practice: object  (missing%: 0.0  unique: 18)
 - Agreement Type: object  (missing%: 0.01  unique: 10)
 - Lending Instrument Type: object  (missing%: 0.05  unique: 4)
 - Approval FY: float64  (missing%: 0.2  unique: 58)
 - Final Closing FY: float64  (missing%: 32.41  unique: 50)
 - Evaluation Type: object  (missing%: 0.0  unique: 7)
 - Outcome: object  (missing%: 1.2  unique: 7)
 -